# 구조적 API 기본연산

## 데이터 프레임 생성

In [1]:
df = spark.read.format("json")\
.load('file:///home/ubuntu/study/spark/data/flight-data/json/2015-summary.json')

## 5.1 스키마

#### 스키마 확인

In [2]:
# 스키마는 여러개의 structfield 타입으로 이루어진 structType 
# StructField(name, type, nan값여부)
# 모든 type은 spark의 데이터 타입과 일치해야한다.
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

#### 스키마 생성

In [3]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(),True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(),True),
    StructField("count", LongType(), False, metadata={'hello':'world'})])

In [4]:
df = spark.read.format('json').schema(myManualSchema)\
.load('file:///home/ubuntu/study/spark/data/flight-data/json/2015-summary.json')

## 5.2 컬럼과 표현식

스파크의 컬럼은 표현식을 사용해 레코드 단위로 계산한 값을 단순하게 나타내는 논리적인 구조이다.  
컬럼의 실제값을 얻으려면 로우가 필요하고, 로우를 얻기 위해서는  DataFrame이 필요하다.  
컬럼 내용을 수정하려면 반드시 DataFrame의 스파크 트랜스포메이션을 이용해야한다.

### 5.2.1 컬럼

#### 컬럼 생성

In [5]:
from pyspark.sql.functions import col, column

col("someColumnName")
column('someColumnName')

Column<b'someColumnName'>

#### 명시적 컬럼 참조  
col 메서드를 사용해 명시적으로 컬럼을 정의하면, 스파크 분석기 실행단계에서 컬럼 확인 절차를 생략한다.

In [6]:
df.column("count")

AttributeError: 'DataFrame' object has no attribute 'column'

### 5.2.2 표현식

#### 표현식으로 컬럼 표현 
표현식은 DataFrame의 레코드의 여러 값에 대한 트랜스포메이션의 집합을 의미한다.   
여러 컬럼을 입력받아 레코드를 읽고 단일 값들을 출력하는 함수라고 생각하면 된다. 

In [7]:
from pyspark.sql.functions import expr

expr("((((col(someCol) + 5) * 200) - 6) < otherCol)")

Column<b'((((col(someCol) + 5) * 200) - 6) < otherCol)'>

#### DataFrame 컬럼에 접근하기

In [8]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## 5.3 레코드와 로우  
스파크 DataFrame 에서 하나의 레코드는 Row 객체로 표현된다.

In [9]:
# 첫번째 row 확인하기
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

### 5.3.1 로우 생성하기

In [10]:
from pyspark.sql import Row

myRow = Row("Hello", None, 1, False)

In [11]:
myRow[0], myRow[2]

('Hello', 1)

## 5.4 DataFrame의 트랜스포메이션  
주요작업
* 로우나 컬럼 추가  
* 로우나 컬럼 제거  
* 로우를 컬럼으로 변환하거나, 그 반대  
* 컬럼값을 기준으로 로우 순서 변경  

### 5.4.1 DataFrame 생성하기  

In [12]:
df = spark.read.format('json').load('file:///home/ubuntu/study/spark/data/flight-data/json/2015-summary.json')
df.createOrReplaceTempView("dfTable")

#### 직접 데이터 베이스 생성하기

In [13]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("col", StringType(), True),
    StructField("names", LongType(), False),
])

myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



### 5.4.2 select 와 Expr  
컬럼이나 표현식을 사용하는 select 메서드 

In [14]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [15]:
# 파이썬으로 작동하기 때문에 큰 따옴표와 작은 따옴표 둘다 가능 
df.select("DEST_COUNTRY_NAME", 'ORIGIN_COUNTRY_NAME').show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [16]:
from pyspark.sql.functions import expr, col, column

df.select(
    expr("DEST_COUNTRY_NAME"),
    col("ORIGIN_COUNTRY_NAME"),
    column("count")
).show(2)

# column 객체와 문자열을 섞어서 사용하면 complie 에러가 발생하니까 주의!!

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [17]:
# as 를 alias로 사용가능
df.select(expr("DEST_COUNTRY_NAME as destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [18]:
# alias를 표현식뒤에 붙여서 표현식 안의 as 처럼 사용가능
df.select(expr("DEST_COUNTRY_NAME as destination").alias("sleepy")).show(2)

+-------------+
|       sleepy|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



#### selectExpr  
select 와 expr 을 자주 사용하기때문에 selectExpr 이라는 메서드가 존재  
매우 효율적이며 spark의 진정한 능력!! 새로운 데이터를 생성하는 복잡한 표현식을 간단하게 만드는 도구  

In [19]:
df.selectExpr("DEST_COUNTRY_NAME as destination","DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|  destination|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [20]:
df.selectExpr("*", "DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME as withinCountry").show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [21]:
# selectExpr 을 이용해 집계함수를 사용할 수 있다.
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



### 5.4.3 스파크 데이터 타입으로 변경하기 
lit(literal) 함수를 이용하여 스파크 데이터 타입으로 변경가능

In [25]:
from pyspark.sql.functions import lit
# 이때 함수를 첨가하거나 할때는 selectExpr 함수 불가능
df.select(expr("*"), lit(1).alias("One")).show(2)


+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



### 5.4.4 컬럼 추가하기  

DataFrame 의 withColumn 메서드를 이용하여 공식적으로 신규 칼럼을 추가할 수 있다.  
인수는 컬럼이름, 컬럼값들

In [29]:
df.withColumn("number One", lit(1)).show(2)

+-----------------+-------------------+-----+----------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|number One|
+-----------------+-------------------+-----+----------+
|    United States|            Romania|   15|         1|
|    United States|            Croatia|    1|         1|
+-----------------+-------------------+-----+----------+
only showing top 2 rows



In [32]:
df.withColumn('withinCountry', expr('DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME')).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [34]:
## withColumn 메서드를 이용하여 칼럼명을 변경할 수 있다
df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

### 5.4.5 컬럼명 변경하기  
withColumnRenamed 메서드를 이용하여 컬럼병을 변경할 수 있다.  
인수는 변경대상, 변경결과

In [36]:
df.withColumnRenamed("DEST_COUNTRY_NAME", 'Dest').show(2)

+-------------+-------------------+-----+
|         Dest|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
+-------------+-------------------+-----+
only showing top 2 rows



### 5.4.6 예약 문자와 키워드  
공백이나 하이픈 같은 예약 문자는 컬럼명에 사용할 수 없다.   
파이썬 문법을 이용하여 보내는 경우에는 괜찮지만 selectExpr과 같이 표현식을 이용하는 경우에는 spark 의 예약문자에 해당하기 때문에 사용할 수 없다.  
이경우 백틱 문자 ` 를 이용하여 이스케이핑 가능

In [37]:
## 필요한 경우
dfWithLongColName = df.withColumn(
"This Long Column-Name",
    expr("DEST_COUNTRY_NAME")
)

In [39]:
## 필요없는 경우
dfWithLongColName.selectExpr(
    "`This Long Column-Name`",
"`This Long Column-Name` as `new col`").show(2)

+---------------------+-------------+
|This Long Column-Name|      new col|
+---------------------+-------------+
|        United States|United States|
|        United States|United States|
+---------------------+-------------+
only showing top 2 rows



### 5.4.7 대소문자 구분   
기본적으로 스파크는 대소문자를 가리지 않는다. 다음과 같은 설정을 사용해 스파크에서 대소문자를 구분하게 할 수 있다.

In [41]:
# -- SQL
# set spark.sql.caseSentitive true

### 5.4.8 컬럼 제거하기

In [42]:
df.drop("DEST_COUNTRY_NAME").columns

['ORIGIN_COUNTRY_NAME', 'count']

In [43]:
df.drop("DEST_COUNTRY_NAME", "origin_COUNTRY_NAME").columns

['count']

### 5.4.9 컬럼의 데이터 타입 변경하기  
cast 메서드를 이용하여 데이터 타입을 변경할 수 있다

In [46]:
df.withColumn("count2", col("count").cast("string"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: string]

### 5.4.10 로우 필터링 하기  
참과 거짓을 판별하는 표현식을 만들어야 한다.
가장 일반적인 방법은 문자열 표현식이나, 컬럼 연산을 이용한 표현식을 만드는 것이다.  
DataFrame의 where 과 filter 함수로 필터링 가능하며, 두 메서드의 기능은 동일하다. 

In [53]:
df.filter(col('count') < 2).show(1)
df.where("count <  2").show(1)
df.where(col('count') < 2).where("DEST_COUNTRY_NAME == \"Croatia\"").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 1 row

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 1 row

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
+-----------------+-------------------+-----+



### 5.4.10 고유한 로우 얻기  
고윳값이나 중복되지 않는 값을 얻으려는 연산을 자주하는데 고윳값을 얻으려면 하나 이상의 컬럼을 사용해야한다. 
distinct 메서드를 이용하여 교윳값을 찾을 수 있다.  

In [54]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

In [56]:
df.select("DEST_COUNTRY_NAME").distinct().count()# count 대신 .show()는 다 보여줌

132

### 5.4.12 무작위 샘플 만들기

In [65]:
seed = 5
wihtReplacement = False # 복원 비복원
fraction = 0.5 # 무작위 샘플의 크기
df.sample(wihtReplacement, fraction, seed).count()

126

### 5.4.13 임의 분할하기  
DataFrame을 임의 크기로 반할할때 유용하게 사용된다. 학습, 검증, 테스트에 주로이용

In [66]:
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count() > dataFrames[1].count()

False

### 5.4.14 로우 합치기와 추가하기  
DataFrame 은 불변성을 가지기 때문에 레코드를 추가하는 것을 불가능하다. 
DataFrame 에 레코드를 추가하려면, 두개의 DataFrame을 통합해야한다. 

In [71]:
from pyspark.sql import Row

schema = df.schema
newRows = [
    Row("sehyeon", "hyeyeon", 1),
    Row("hyeyeon", "sehyeon", 1)
]

# 만든 데이터를 spark.sparkContext.parallezie를 이용해 펴줘야함 
paralleizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(paralleizedRows, schema)

newDF.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          sehyeon|            hyeyeon|    1|
|          hyeyeon|            sehyeon|    1|
+-----------------+-------------------+-----+



In [72]:
df.union(newDF).where('count = 1').where(col("ORIGIN_COUNTRY_NAME") == 'hyeyeon').show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          sehyeon|            hyeyeon|    1|
+-----------------+-------------------+-----+



### 5.4.15 로우 정렬하기  
sort와 orderBy 메서드를 사용해 DataFrame의 최댓값 혹은 최솟값이 상단에 위치하도록 가능  
두 메서드는 완전히 동일한 방식으로 작동한다.

In [77]:
from pyspark.sql.functions import desc, asc

df.orderBy(expr("count desc")).show(1)
df.sort(col('count').desc(), col("DEST_COUNTRY_NAME").asc()).show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 1 row

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
+-----------------+-------------------+------+
only showing top 3 rows



### 5.4.16 로우수 제한하기 
limit 메서드를 이용해 row 수를 제한할 수 있다. 

In [79]:
df.limit(1).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
+-----------------+-------------------+-----+



In [81]:
df.orderBy(col('count').asc()).limit(2).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



### 5.4.17 repartition과 coalesce  
또 다른 최적화 기법은 자주 필터링 하는 컬럼을 기준으로 데이터를 분할하는 것이다.  
파티셔닝 스키마와 파티션 수를 포함해 클러스터 전반의 물리적 데이터 구성을 제어할 수 있다.  
repartition 메서드를 호출하면 무조건 전체 데이터를 셔플한다.  
향후에 사용할 파티션 수가 현재보다 많거나 컬럼을 기준으로 할때만 사용 !! 

In [82]:
df.rdd.getNumPartitions()

1

In [83]:
# 파티션 개수와 자주 사용하는 컬럼을 인자로 전달해 재파티션 가능 
df.repartition(5, col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [84]:
# coalesce 를 이용하여 셔플하지 않고 파티션을 병합할 수 있다. 
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2).rdd.getNumPartitions()

2

### 5.4.18 드라이버로 로우 데이터 수집하기  
스파크는 드라이버에서 클러스터 상태정보를 유지한다. 
로컬 환경에서 데이터 다루려면 드라이버로 데이터를 수집해야 한다.  
몇가지 예제 collect 같은경우 DataFrame의 모든 데이터를 수집한다.  
take 나 show 도 지정한 개수만큼 합쳐서 출력한다.  ㅡ

In [88]:
collectDF = df.limit(10)
collectDF.take(5)
collectDF.show()
collectDF.show(5, False) # ?안보여주네? 뭔소용인거지
collectDF.collect()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India         

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]